# Semi-discrete optimal transport and application to computational fluid dynamics

## Semi-discrete optimal transport

### Presentation of the toolbox

$\newcommand{\Lag}{\mathrm{Lag}}
\newcommand{\Ker}{\mathrm{Ker}}
\newcommand{\abs}[1]{\left| #1 \right|}
\newcommand{\sca}[2]{\langle#1|#2\rangle}
\newcommand{\Class}{\mathcal{C}}
\newcommand{\Wass}{\mathrm{W}}
\newcommand{\D}{\mathrm{D}}
\newcommand{\B}{\mathrm{B}}
\newcommand{\one}{\textbf{1}}
\newcommand{\hdots}{\dots}
\newcommand{\dd}{\mathrm{d}}
\newcommand{\Rsp}{\mathbb{R}}
\newcommand{\eps}{\varepsilon}
\newcommand{\nr}[1]{\|#1\|}
$

We study here the optimal transport problem between a density
$\rho$ supported on a convex domain $\Omega$ of $\Rsp^2$ and a finitely supported
measure $\nu = \sum_{i\in I} \nu_i\delta_{y_i}$, where $I =
\{1,\hdots,N\}$. This problem can be recast as finding a Kantorovich
potential (a vector $\psi\in \Rsp^N$) satisfying the non-linear system
of equations

$$ \tag{P}\qquad \forall i\in I, G_i(\psi) = \nu_i, $$

where

$$\begin{aligned}
  &G_i(\psi) = \rho(\Lag_i(\psi)) := \int_{\Lag_i(\psi)} \rho(x) \dd x,
  \\ &\Lag_i(\psi) = \{x \in \Omega\mid \forall j\in I, c(x,y_i) + \psi_i
  \leq c(x,y_j) + \psi_j \}.
\end{aligned}$$

By default, we consider the cost $c(x,y) = \nr{x - y}^2$.  When the set $Y$ is ambiguous, or
we want to consider another cost $c$, we will sometime use the heavier
notation $\Lag^c_i(y_1,\hdots,y_N;\psi_1,\hdots,\psi_N)$ for denoting
the $i$th Laguerre cell.  The set $\Lag_i(\psi) \subseteq P$ is called
the Laguerre cell of the $i$th point, and the collection
$(\Lag_i(\psi))_{1\leq i\leq N}$ is called the Laguerre
diagram.  We provide a wrapper around our code PySDOT to
simplify the computations on Laguerre diagrams (areas and
barycenters) when $\rho$ is constant.

In [1]:
# uncomment the following line if you're using colab.research.google
# !pip install pysdot

In [56]:
# imports
from pysdot.domain_types import ConvexPolyhedraAssembly
from pysdot.domain_types import ScaledImage
from pysdot import PowerDiagram
from IPython.display import clear_output
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
import numpy as np
import time


# constructs a square domain, to be passed to the laguerre_* functions
def make_square(box=[0, 0, 1, 1]):
    domain = ConvexPolyhedraAssembly()
    domain.add_box([box[0], box[1]], [box[2], box[3]])
    return domain

# constructs a square domain, to be passed to the laguerre_* functions
def make_image(img, box=[0, 0, 1, 1]):
    img = img / ((box[2] - box[0]) * (box[3] - box[1]) * np.mean(img))
    return ScaledImage([box[0], box[1]], [box[2], box[3]], img)

# displays the Laguerre diagram
# optional arguments: disp_positions=True/False (display the points Y),
#                     disp_ids=True/Falsue (show the number of each cell),
#                     disp_centroids=True/False (show the centroid of each cell)
#                     disp_arrows=True/False (display arrows beween points and centroids)
#                     hide_after=optionnal id of the first cell to be hidden. < 0 means no hide
def laguerre_draw(domain, Y, psi, disp_ids=False, disp_positions=True, disp_centroids=False, 
            disp_arrows=False, hide_after=-1):
    if type(psi)==list:
        cpsi = [ -p for p in psi ]
    else:
        cpsi = -psi
    display(PowerDiagram(Y, cpsi, domain).display_jupyter(
        disp_ids=disp_ids,
        disp_positions=disp_positions,
        disp_centroids=disp_centroids,
        disp_arrows=disp_arrows,
        hide_after=hide_after
    ))
    

# computes the areas of the Laguerre cells intersected with the domain, and returns it as an array
# if der = True, also returns a sparse matrix representing the Jacobian of the areas with respect to psi
def laguerre_areas(domain, Y, psi, der=False):
    pd = PowerDiagram(Y, -psi, domain)
    if der:
        N = len(psi)
        mvs = pd.der_integrals_wrt_weights()
        return mvs.v_values, csr_matrix((-mvs.m_values, mvs.m_columns, mvs.m_offsets), shape=(N, N))
    else:
        return pd.integrals()

# computes the centroid of the Laguerre cells intersected with the domain, and returns it as a Nx2 array
def laguerre_centroids(domain, Y, psi):
    return PowerDiagram(Y, -psi, domain).centroids()

### Numerical resolution of semi-discrete optimal transport

Equation (P) can be written as $G(\psi) = \nu$, where
$G(\psi) = (G_1(\psi),\hdots,G_N(\psi))\in\Rsp^N$ and where $ \nu =
(\nu_1,\hdots,\nu_N)\in\Rsp^N$.


**Q1.** The applet below allows to change the value of $\psi$ interactively. Try to solve the case $N=5$ by hand to get an understanding of what happens. 

In [57]:
# case N=5
from ipywidgets import interact, FloatSlider

N = 5
domain = make_square()
Y = np.array([[.15,.15],[.7,.2], [.4,.8], [.3,.5], [.7,.7]])
    
def disp(**kwargs):
    psi = np.array([p for p in kwargs.values()])
    print("areas=", laguerre_areas(domain, Y, psi))
    laguerre_draw(domain, Y, psi, disp_positions=True, disp_ids=True, disp_centroids=False)
    
interact(disp, **{"psi%i"%i : FloatSlider(min=-.1,max=.1,step=0.0001,value=0.0) for i in range(N)});

interactive(children=(FloatSlider(value=0.0, description='psi0', max=0.1, min=-0.1, step=0.0001), FloatSlider(…

**Q2.** Start with $N \in \{5,10,20\}$ random points in the square $[0,\frac{1}{2}]^2$, $\Omega = [0,1]^2$, and $\nu = (\frac{1}{N},\hdots,\frac{1}{N})\in\Rsp^N$. Then try to use the function `scipy.optimize.fsolve`  to solve equation (P).

In [74]:
from scipy.optimize import fsolve

N = 10
np.random.seed(0)
domain = make_square()
Y = .5 * np.random.rand(N, 2)
nu = np.ones(N) / N

### BEGINNING OF YOUR CODE
def func(psi):
    return laguerre_areas(domain, Y, psi) - nu

start = time.time()
psi_0 = np.zeros(N)
psi = fsolve(func, psi_0)
end = time.time()
print("Duration:", end-start)
### END OF YOUR CODE

laguerre_draw(domain, Y, psi, disp_arrows=True)
print(laguerre_areas(domain, Y, psi))

Duration: 0.017006874084472656


<IPython.core.display.Javascript object>

[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


**Q3.** Check numerically that $G(\psi+e) = G(\psi)$, where $e =
(1,\hdots,1)\in\Rsp^N$ (the Laguerre cells are invariant by addition
  of a constant to each coordinates of $\psi$), and that $e \in \Ker
  \D G(\psi)$, so that $\D G(\psi)$ is never invertible.

In [75]:
N = 10
np.random.seed(0)
domain = make_square()
#Y = .5 * np.random.rand(N, 2)
### BEGINNING OF YOUR CODE
# Translation invariance
psi_0 = np.zeros(N)
psi = fsolve(func, psi_0)
e = np.ones(N)
print("||G(psi + e) - G(psi)||_2 =", np.linalg.norm(laguerre_areas(domain, Y, psi)-\
                                                    laguerre_areas(domain, Y, psi + e)))
# Singularity of DG
_, DG = laguerre_areas(domain, Y, psi, der=True)
print("||DG(e)|| =", np.linalg.norm(DG.dot(e)))
### END OF YOUR CODE

||G(psi + e) - G(psi)||_2 = 5.846815929450271e-16
||DG(e)|| = 1.3232745389513375e-15


We now turn to a much more efficient Newton's method for solving (P). Since $\D
G(\psi)$ is not invertible because of the invariance by addition of a
constant, we will fix $\psi_N = 0$. In practice, we simply remove the
last unknown ($\psi_N$) and also the last equation of the system
(which is then redundant, as $\sum_i G_i(\psi) = 1$), and define

$$\begin{aligned} F: \Rsp^{N-1} &\to \Rsp^{N-1}
  \\ \psi' = (\psi'_1,\hdots,\psi'_{N-1}) &\mapsto
  (G_1(\psi'_1,\hdots,\psi'_{N-1},0),\hdots,G_{N-1}(\psi'_1,\hdots,\psi'_{N-1},0)).
\end{aligned}
$$

The iterates in Newton's algorithm are denoted with an exponent,
${\psi'}^{(0)},\hdots,{\psi'}^{(k)}\in\Rsp^{N-1}$.  Linearizing the equation
$F({\psi'}^{(k)} + d^{(k)}) = \nu'$ (where $\nu' = (\nu_1,\hdots,\nu_{N-1})$, we get

\begin{equation} \tag{N}
  \begin{cases}
  \D F({\psi'}^{(k)})\cdot d^{(k)} = \nu' - F(\psi^{(k)}) \\
  {\psi'}^{(k+1)} = {\psi'}^{(k)} + d^{(k)}
\end{cases}
\end{equation}


**Q4.** Compute $\D F(\psi_1,\hdots,\psi_{N-1})$ explicitly and prove that if $\Ker \D
  G(\psi_1,\hdots,\psi_{N-1},0) = \Rsp e$, then $\D F(\psi_1,\hdots,\psi_{N-1})$ is
  invertible. Implement Newton's method for solving (P). Test
  it for $N \in \{10,100,1000\}$ random iid points in $[0,1]^2$, $\Omega =
  [0,1]^2$ and $\nu = \frac{1}{N} e \in \Rsp^N$.

In [80]:
# use spsolve to solve the linear system
from scipy.sparse.linalg import spsolve

N = 10
np.random.seed(0)
domain = make_square()
#Y = np.random.rand(N, 2)
nu = np.ones(N)/N
nu_prime = nu[:-1]

### BEGINNING OF YOUR CODE
# Define F and DF
def F_and_DF(psi_prime):
    psi = np.append(psi_prime, np.array([0]))
    G, DG = laguerre_areas(domain, Y, psi, der=True)
    return G[:-1], DG[:-1, :-1]

# Newton's algorithm
start = time.time()
psi_prime = np.zeros(N-1)
conv = False
tol = 1e-4
while not conv:
    F_k, DF_k = F_and_DF(psi_prime)
    d = spsolve(DF_k, nu_prime - F_k)
    psi_prime = psi_prime + d
    conv = (np.linalg.norm(d)<tol)
end = time.time()
print("Duration:", end-start)
psi = np.append(psi_prime, np.array([0]))
### END OF YOUR CODE

laguerre_draw(domain, Y, psi, disp_arrows=True)
print(laguerre_areas(domain, Y, psi))

Duration: 0.00926065444946289


<IPython.core.display.Javascript object>

[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


A sufficient condition for the invertibility of $\D F(\psi')$ is that $\psi'\in \mathcal{K}$ where

$$\mathcal{K} = \{ \psi'\in\Rsp^{N-1} \mid \forall i
\in\{1,\hdots, N-1\}, \quad F_i(\psi')  > 0 \}.
$$

In practice, this condition is ensured throught a simple
backtracking procedure: one takes ${\psi'}^{(k+1)} = {\psi'}^{(k)} +
t^{(k)} d^{(k)}$ where $t^{(k)} \in (0,1]$ is chosen small enough so
  that ${\psi'}^{(k+1)} \in \mathcal{K}$. This procedure is
  implemented in the function `optimal_transport(P,Y,nu,psi0)` provided below. The last 
  parameter `psi0`$\in \Rsp^N$ is optional, default to $0\in \Rsp^N$, and should be chosen so that
  
  \begin{equation}
    \forall i\in I, G_i(\psi^{(0)}) > 0.
  \end{equation}  

In [81]:
def optimal_transport(domain, Y, nu, psi0=None, verbose=False, maxerr=1e-6, maxiter=20):
    if psi0 is None:
        psi0 = np.zeros(len(nu))
        
    def F(psip):
        
        g,h = laguerre_areas(domain, Y, np.hstack((psip,0)), der=True)
        return g[0:-1], h[0:-1,0:-1]
    
    psip = psi0[0:-1] - psi0[-1]
    nup = nu[0:-1]
    g,h = F(psip)
    for it in range(maxiter):
        err = np.linalg.norm(nup - g)
        if verbose:
            print("it %d: |err| = %g" % (it, err))
        if err <= maxerr:
            break
        d = spsolve(h, nup - g)
        t = 1.
        psip0 = psip.copy()
        while True:
            psip = psip0 + t * d
            g,h = F(psip)
            if np.min(g) > 0:
                break
            else:
                t = t/2
    return np.hstack((psip,0))

### Application to optimal quantization


In optimal quantization, one wishes to approximate a probability density $\rho$ over a domain $\Omega\subseteq \Rsp^d$ by a finitely supported measure of the form $\mu = \frac{1}{N} \sum_{1\leq i\leq N} \delta_{y_i}$ in an optimal way, i.e. so as to minimize the quantization energy 

$$ \mathcal{D}(y_1,\hdots,y_N) = N \Wass_2^2(\rho, \frac{1}{N} \sum_{1\leq i\leq N} \delta_{y_i}) $$

As shown in the course 

$$ \nabla_{y_i} \mathcal{D}(y_1,\hdots,y_N) = b(y_1,\hdots,y_N) $$ 

where  $b_i(y_1,\hdots,y_N)$ is defined as follows:
- first, compute $\psi\in\Rsp^N$ the Kantorovich potential in
    the optimal transport problem between the $\rho$ on $\Omega$ and
    $\nu = \frac{1}{N}\sum_{i\in I} \delta_{y_i}$.
- $b_i(y_1,\hdots,y_N)$ is then the centroid of the Laguerre
    cell $\Lag_i(y_1,\hdots,y_N;\psi_1,\hdots,\psi_N)$ weighted by the density $\rho$.

**Q5.**  Using the parameter `disp_arrow` of the function `laguerre_draw`,  observe the paths from the diracs to the centroids when
  - $\Omega = [-\frac{1}{2},\frac{1}{2}]$
  - $\rho = \one_{\Omega}$
  - $y_1,\hdots,y_N$ are random iid in $\Omega$ and the in $\frac{1}{2}\Omega$)

In [86]:
# use spsolve to solve the linear system
from scipy.sparse.linalg import spsolve

N = 10
np.random.seed(2)
domain = make_square()
Y = np.random.rand(N, 2)
nu = np.ones(N)/N
nu_prime = nu[:-1]

### BEGINNING OF YOUR CODE
# Define F and DF
def F_and_DF(psi_prime):
    psi = np.append(psi_prime, np.array([0]))
    G, DG = laguerre_areas(domain, Y, psi, der=True)
    return G[:-1], DG[:-1, :-1]

# Newton's algorithm
start = time.time()
psi_prime = np.zeros(N-1)
conv = False
tol = 1e-10
while not conv:
    F_k, DF_k = F_and_DF(psi_prime)
    d = spsolve(DF_k, nu_prime - F_k)
    psi_prime = psi_prime + d
    conv = (np.linalg.norm(d)<tol)
    psi = np.append(psi_prime, np.array([0]))
    laguerre_draw(domain, Y, psi, disp_arrows=True)
end = time.time()
print("Duration:", end-start)
### END OF YOUR CODE


print(laguerre_areas(domain, Y, psi))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Duration: 0.042603492736816406
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


In [89]:
np.random.seed(0)
### BEGINNING OF YOUR CODE
### END OF YOUR CODE

A simple algorithm for optimal quantization consists in performing gradient descent on $\mathcal{D}$, e.g. 

$$ \begin{cases} Y^{(0)} \in \Rsp^{2 N} \\
Y^{(k+1)} = Y^{(k)} - \tau \nabla \mathcal{D}(Y^{(k)})
\end{cases} $$

**Q6.** Implement and test this algorithm for $\rho = \one_{\Omega}$, starting with $Y^{(0)}$ a family of i.i.d random points in $\Omega$.

Check the formation of hexagonal patterns (as predicted by a theorem of L. Fejes Toth), by displaying the Laguerre diagram after the last iteration.

In [149]:
def optimal_quantization(domain, Y, tau=.1, niter=50, nstep_to_disp=0):
    d = np.arange(nstep_to_disp) * (niter - 1) // (nstep_to_disp - 1 + (nstep_to_disp==1))
    for i in range(niter):
        m = domain.measure() / Y.shape[0]
        psi = optimal_transport(domain, Y, m * np.ones(Y.shape[0]), verbose=False)
        #if i in d:
        clear_output(True)
        laguerre_draw(domain, Y, psi, disp_centroids=True)
        if i + 1 == niter:
            break
        ### BEGINNING OF YOUR CODE
        step = Y - np.multiply(laguerre_centroids(domain, Y, psi),\
                               np.expand_dims(laguerre_areas(domain, Y, psi), axis=-1))
        Y = (Y - tau*step).copy()
        #print(np.linalg.norm(step))
        ### END OF YOUR CODE
    return Y, psi
  
### BEGINNING OF YOUR CODE
N = 50
Y = np.random.rand(N, 2)
Y, psi = optimal_quantization(domain, Y, tau=.05, niter=1500, nstep_to_disp=1)
laguerre_draw(domain, Y, psi)
### END OF YOUR CODE

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Q7** Test `optimal_quantization` for $\rho = \frac{1}{Z}\exp(-2 \nr{\cdot}^2) \one_{\Omega}$, where $\Omega = [-\frac{1}{2},\frac{1}{2}]$ and $Z$ is a normalizing constant
(NB: one can implement a non-constant density by using the function `make_image(img,box)`, which constructs a normalized piecewise constant probability density from an image and a bounding box.) 

In [150]:
domain = make_square([-.5, -.5, .5, .5])


In [ ]:
### BEGINNING OF YOUR CODE
### END OF YOUR CODE

### Application to  Euler's equations for incompressible fluids


Given a domain $\Omega \subseteq \Rsp^2$, an initial velocity field $u_0:
\Omega\to\Rsp$ and an initial configuration of points $Y(0) =
(y_1(0),\hdots,y_N(0))\in\Rsp^d$, we consider the following system of
ODE as an approximation of Euler's equations for incompressible
fluids, where $b_i$ is defined above:

\begin{equation}  \tag{Euler}
  \begin{cases}
    \ddot{y}_i(t) = \frac{1}{\eps} (b_i(y_1(t),\hdots,y_N(t)) - y_i(t)) \\
    \dot{y}_i(0) = u_0(Y_i(t)) \\
  \end{cases}
\end{equation}

**Q8.** Implement the symplectic Euler scheme to solve the ODE (Euler):

  $$ \begin{cases}
    u_i^{(n+1)} = u_i^{(n)} + \frac{\tau}{\eps^2}(b_i(y_1,\hdots,y_n) - y_i)\\
    y_i^{(n+1)} = y_i^{(n)} + \tau u_i^{(n+1)} \\  
    [y_1^{(0)},\hdots,y_N^{(0)}] = \texttt{barycenters(rand(N,2)-.5)} \\
    v_i^{(0)} = u_0(y_i^{(0)}). 
  \end{cases}
  $$
  
  As it can be quite tricky to select the parameters, we suggest: 
  - $\Omega = [-\frac{1}{2}, \frac{1}{2}]^2$, 
  - $N=400$, $\eps = 0.05$, $\tau = 0.01$
  - $u_0(y_1,y_2) = (-\cos(\pi y_1) \sin(\pi y_2), \sin(\pi y_1) \cos(\pi y_2)).$

In [ ]:
N = 300
np.random.seed(0)
domain = make_square([-.5,-.5,.5,.5])

Y, psi = optimal_quantization(domain, -.5 + np.random.rand(N, 2))
U = np.zeros_like(Y)
U[:,0] = -np.cos(np.pi * Y[:, 0]) * np.sin(np.pi * Y[:, 1])
U[:,1] = np.sin(np.pi * Y[:, 0]) * np.cos(np.pi * Y[:, 1])
tau = 0.002
eps = 0.05

# Symplectic Euler
hist_Y, hist_psi = [], []
for i in range(200):
    ### BEGINNING OF YOUR CODE
    ### END OF YOUR CODE

    if i % 20 == 0:
        clear_output(True)
        laguerre_draw(domain, Y, psi)
    hist_psi.append(psi)
    hist_Y.append(Y)

clear_output(True)
laguerre_draw(domain, hist_Y[::5], hist_psi[::5])

## Partial optimal transport and crowd motion

### Partial optimal transport 

As before, we assume that $\rho$ is a density on a domain
$\Omega\subseteq \Rsp^2$ and $\nu = \frac{\alpha}{N}\sum_{i\in
  I}\delta_{y_i}$. However, in _partial optimal transport_, one
assumes that the total mass of $\nu$ is strictly less than the mass of
$\rho$, i.e. $\alpha < \rho(P)$. The effect of the constant $\alpha$
is displayed in figure below. It is possible to turn
partial optimal transport into classical optimal transport as follows:
- introduce a fictive point $y_{\infty}$ which will receive the
    left-over mass. Set $\widetilde{I} = I \cup \{+\infty\} =
    \{1,\hdots,N,+\infty\}$ and $\widetilde{\nu} = \alpha
    \delta_{y_\infty} + \nu$ ;
- introduce a modified cost $\widetilde{c}(x,y_i) = \nr{x -
    y_i}^2$ if $i\in I$ and $c(x,y_\infty) = 0$. 
    
With this cost, it is free to move mass to the fictive point $y_\infty$.
Then, as in classical optimal transport, one wishes to solve the
non-linear system

$$\forall i\in I, \widetilde{G}_i(\widetilde{\psi}) =
\nu_i$$

where 
- $\widetilde{\psi} = (\psi_1,\hdots,\psi_N,\psi_\infty)
\in \Rsp^{N+1}$, 
- $\widetilde{G}_i(\widetilde{\psi}) =
\mathrm{area}(\widetilde{\Lag}_i(\psi))$ and 
$$\begin{align*}
  \widetilde{\Lag}_i(\psi_1,\hdots,\psi_N,\psi_\infty) &=
  \Lag_i^{\tilde{c}}(y_1,\hdots,y_N,y_\infty,\psi_1,\hdots,\psi_N,0)
  \\ &=\{ x\in P \mid \forall j\in \widetilde{I}, \widetilde{c}(x,y_i)
  + \psi_j \leq \widetilde{c}(x,y_j) + \psi_j\}
\end{align*}$$

**Q9.** Check that for all $i\in I$, 
  $\widetilde{\Lag}_i(\widetilde{\psi}) = \Lag_i(y_1,\hdots,y_N;\psi)
  \cap \B(y_i, \sqrt{\psi_\infty-\psi_i}), $ where $\Lag_i(\psi)$ is
  the usual Laguerre cell (as defined in the first part) and $\B(y,r)$ is the ball centered at $y$ with radius $r$.
   
  
<table>
    <tbody>
        <tr>
            <td> <img src="https://www.dropbox.com/s/8p625w5x8trah0r/partial-0.jpg?dl=1" width="100%" /> </td>
            <td> <img src="https://www.dropbox.com/s/i2g929h4a3a7kbv/partial-1.jpg?dl=1" width="100%" /> </td>
            <td> <img src="https://www.dropbox.com/s/pgqa76dhpgbuo6y/partial-2.jpg?dl=1" width="100%" /> </td> 
            <td> <img src="https://www.dropbox.com/s/77r977gvv8a7oyj/partial-3.jpg?dl=1" width="100%" /> </td>
        </tr>
    </tbody>
    <caption> Partial optimal transport between the measure $\nu =
    \frac{\alpha}{N} \sum_{1\leq i\leq N} \delta_{y_i}$ and $\rho =
    \one_{[-1,1]^2}$. The first figure displays the points $y_i$ and
    the next figures display the Laguerre cells obtained for $\alpha
    =1,2,3$ respectively. The Laguerre cells in multiple colors
    correspond the portion of the domain $[-1,1]^2$ that will be
        transported towards $\nu$. </caption>
</table>


Test `partial_optimal_transport` (given below) with total masses equal to 1.5, 1.0 and 0.5 (in a larger domain). The function `partial_optimal_transport` returns an object with the class `PowerDiagram`. This class contains a few member functions such as 
- `display_jupyter`, with the same arguments as the global function you have already used
- `centroids`, which returns the centroids of the Laguerre cells $\widetilde{\Lag}_i$.


In [ ]:
from pysdot.radial_funcs import RadialFuncInBall
from pysdot import OptimalTransport

def partial_optimal_transport(domain, Y, nu, psi0 = None):
    if psi0 == None:
        psi = -np.sqrt(nu)
    else:
        psi = -psi0.copy()
    ot = OptimalTransport(Y, -psi, domain, radial_func=RadialFuncInBall())
    ot.set_masses(nu)
    ot.adjust_weights()
    return ot.pd

  
### BEGINNING OF YOUR CODE
### END OF YOUR CODE


### Discretization of a crowd motion model


In this model, the crowd is described by a finite sum of Dirac masses
$\nu(t) = \frac{\alpha}{N}\sum_{1\leq i\leq N} \delta_{y_i}(t)$,
living in a domain $\Omega$ (the ``room'') with area strictly larger than $\alpha$. For
simplicity, we will assume that $\alpha = 1$. They
are subject to two forces: the first force is minus the gradient of a
potential $V$ (corresponding to the fact that they want, for instance
to go towards the exit of the room) and the second force corresponds
to congestion, and is enforced through partial optimal transport:

$$
\begin{equation}  \tag{Crowd}
    \dot{y}_i(t) = -\nabla V(y_i(t)) + \frac{1}{\eps} (c_i(y_1(t),\hdots,y_N(t)) - y_i(t)) \\
\end{equation}
$$

As before, $c_i(y_1(t),\hdots,y_N(t))$ is the barycenter of the $i$th
Laguerre cell in the partial transport between $\nu(t)$ and $\rho$. As
explained in the course, the sum
$$\frac{\alpha}{N} \sum_{i=1}^N
\nr{c_i(y_1(t),\hdots,y_N(t))-y_i(t)}^2 = \min_{\sigma \leq \rho,
  \sigma(P) = \alpha} \mathrm{W}_2^2(\nu(t),\sigma),$$ measures how
far the measure $\nu(t)$ is from satisfying the congestion constraint.

**Q10** Considering $V(x,y) = y$, $\Omega = [0,4]^2$ and a crowd uniformly distributed in the $[1.5,0.5]\times[3,2]$ square, implement and test the (Crowd) model.


In [ ]:
# constants
N = 100 # nb diracs
b = [1.5,0.5,3,2] # initial box
m = (b[2] - b[0]) * (b[3] - b[1]) / N
nu = m * np.ones( N ) # masses
epsilon = 0.5 * m**0.5
timestep = epsilon / 2

# initial positions
Y, psi = optimal_quantization(make_square(b), b[0:2] + 1.5 * np.random.rand(N, 2))

# domain
domain = make_square([0, 0, 4, 4])

# iterations
Y_hist = []
psi_hist = []
for num_time_step in range(100):
    # below, we compute the solution pd of the partial transport problem, and
    # update Y accordingly
    
    ### BEGINNING OF YOUR CODE
    ### END OF YOUR CODE

    Y_hist.append(pd.get_positions().copy())
    psi_hist.append(pd.get_weights().copy())

pd = PowerDiagram(Y_hist, psi_hist, domain, RadialFuncInBall())
display(pd.display_jupyter(disp_ids=False, disp_centroids=False))

**Q11** In the following code, the domain is described by an image ($\Omega$ is the union of white pixels), and the potential $V$ is the geodesic distance to a point in the domain. Complete and test the following implementation:

In [ ]:
# Uncomment the following line if you're using colab.research.google
# !pip install scikit-fmm

In [ ]:
from pysdot.util import FastMarching

# constants
N = 100 # nb diracs
b = 0.33 # initial box size
m = b**2 / N
nu = m * np.ones( N ) # masses
epsilon = 1.0 * m**0.5
timestep = epsilon / 2

# initial positions
Y, psi = optimal_quantization(make_square([0,0,b,b]), b * np.random.rand(N, 2))

# domain
img = np.array([
    [1, 0, 1],
    [1, 1, 1],
    [1, 0, 1]
])
domain = ScaledImage([0, 0], [1, 1], img)

# fast marching
fm = FastMarching(domain, [[0.9,0.1]], 0.01)

# iterations
Y_hist = []
psi_hist = []
for num_time_step in range(100):
    pd = partial_optimal_transport(domain, Y, nu)
    if num_time_step % 2 == 0:
        Y_hist.append(pd.get_positions().copy())
        psi_hist.append(pd.get_weights().copy())

    # grad V
    descent_direction = np.zeros_like(Y)
    for n in range(Y.shape[0]):
        descent_direction[n, :] = fm.grad(Y[n, :], 0.1)
    
    ### BEGINNING OF YOUR CODE
    ### END OF YOUR CODE



pd = PowerDiagram(Y_hist, psi_hist, domain, RadialFuncInBall())
display(pd.display_jupyter(disp_ids=False, disp_centroids=False))

### Approximation of partial optimal transport



Computing the intersection between a ball and a convex polygon
efficiently is rather tedious, but we can resort to an
approximation. Let $Z \subseteq \Omega$ be a finite subset which is
$\eta$-dense in the sense that $\Omega\subseteq \cup_{z\in P} B(z,\eta)$.
We consider an approximation of the previous problem, denoted using
hats instead of tildes. We keep the point at infinity $\hat{I} =
I\cup\{+\infty\}$, $\hat{\nu} = \widetilde{\nu}$ but we set

$$ \hat{c}(x,y_i) = \begin{cases}
  \nr{x - y_i}^2 & \hbox{ if } i\in I\\
  \min_{z\in Z} \nr{z - x}^2 &\hbox{ if } i=+\infty
\end{cases}
$$

We define $\hat{G}_i(\hat{\psi}) = \rho(\hat{\Lag}_i(\psi))$ and
the Laguerre cell  $\hat{\Lag}_i$ accordingly:

$$ \hat{\Lag}_i(\hat{\psi}) = \{ x\in \Omega \mid \forall j\in \hat{I}, \hat{c}(x,y_i) + \psi_j \leq \hat{c}(x,y_j) + \psi_j\}.$$

**Q12.** Prove that $\tilde{c}\leq \hat{c} \leq \tilde{c} + \eta^2$
  (explaining why the problem with $\hat{c}$ is an approximation of
  the problem with $\tilde{c}$). Prove that if $Z = \{z_1,\hdots,z_M\}$, then
  
  \begin{align*}
    \forall i\in\{1,\hdots,N\},~\hat{\Lag}_i(\psi) =
    \Lag_i(y_1,\hdots,y_N, z_1,\hdots,z_N;
    \psi_1,\hdots,\psi_N,\psi_\infty,\hdots,\psi_\infty),
  \end{align*}
  
where  $\psi_\infty$ is repeated $N$ times.
  
In other words, $\hat{\Lag}_i(\psi)$ turns out to be a usual Laguerre
cell, for the quadratic cost. The value of $\hat{G}_i(\psi) =
\rho(\hat{\Lag}_i(\psi))$ and its gradient with respect to $\psi$ can
therefore be easily computed using `laguerre_areas`. We can
now adapt the backtracking Newton algorithm used in the function
`optimal_transport` to the (approximated) partial transport
setting, by setting

$$\begin{aligned} F: \Rsp^N&\to\Rsp^N \\
  (\psi_1,\hdots,\psi_N) &\mapsto (\hat{G}_i(\psi_i,\hdots,\psi_N, 0, \hdots, 0))_{1\leq i\leq N}
\end{aligned}
$$

where $0$ is repeated $N$ times. As in the first part, the matrix $\D F(\psi)$ will be invertible
provided that $\psi\in \mathcal{K}$, where

$$\mathcal{K} = \{ \psi\in\Rsp^{N} \mid \forall i
\in\{1,\hdots, N\},\quad F_i(\psi)  > 0 \}.
$$

**Q13.** Taking inspiration from `optimal_transport`, write a function for partial optimal transport: 
  `partial_transport(domain,Y,nu,Z)` where `domain`,`Y`,`nu` are as before
  (except that $\sum_i \nu_i$ is now strictly less than the area of
 $\Omega$), and $Z$ is a $M\times 2$ matrix whose rows are the
  points in $Z$. Try it in the following setting $Z$ is a $M=20^2$
  uniform grid in $\Omega = [-1,1]^2$, $Y$ is a random iid point cloud with
  cardinal $N=100$. One can display the result by showing the Laguerre
  cells with a color depending on whether they correspond to a point
  in $Y$ or $Z$.

In [ ]:
from pysdot.radial_funcs import RadialFuncInBall
from pysdot import OptimalTransport

N = 100
L = 2.0
np.random.seed(0)
Y = np.random.rand(N,2)
nu = 1.5**2 / N * np.ones(N)
psi = np.zeros(N)
domain = make_square([0,0,L,L])

M = 25
x = np.linspace(0, L, M)
y = np.linspace(0, L, M)
Z = np.vstack([v.flatten() for v in np.meshgrid(x, y)]).transpose()

dlt_psi = None
for it in range(100):
    ### BEGINNING OF YOUR CODE
    ### END OF YOUR CODE

P = np.vstack((Y, Z))
W = np.hstack((psi, np.zeros(M**2)))
laguerre_draw(domain, P, W, hide_after=N)